## Literature

[Ising notes](http://people.math.aau.dk/~kkb/Undervisning/Bayes14/sorenh/docs/ising-notes.pdf)

[A Tutorial on Image Analysis](https://www.researchgate.net/profile/Havard_Rue/publication/245122781_A_Tutorial_on_Image_Analysis/links/551faa860cf2f9c1304df591/A-Tutorial-on-Image-Analysis.pdf?origin=publication_detail)

[Understanding the Metropolis-Hastings Algorithm](http://www.ics.uci.edu/~johnsong/papers/Chib%20and%20Greenberg%20-%20Understanding%20the%20Metropolis-Hastings%20Algorithm.pdf)

In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

In [3]:
def mode(ndarray, axis=0):
    # Check inputs
    ndarray = np.asarray(ndarray)
    ndim = ndarray.ndim
    if ndarray.size == 1:
        return (ndarray[0], 1)
    elif ndarray.size == 0:
        raise Exception('Cannot compute mode on empty array')
    try:
        axis = range(ndarray.ndim)[axis]
    except:
        raise Exception('Axis "{}" incompatible with the {}-dimension array'.format(axis, ndim))

    # If array is 1-D and numpy version is > 1.9 numpy.unique will suffice
    if all([ndim == 1,
            int(np.__version__.split('.')[0]) >= 1,
            int(np.__version__.split('.')[1]) >= 9]):
        modals, counts = np.unique(ndarray, return_counts=True)
        index = np.argmax(counts)
        return modals[index], counts[index]

    # Sort array
    sort = np.sort(ndarray, axis=axis)
    # Create array to transpose along the axis and get padding shape
    transpose = np.roll(np.arange(ndim)[::-1], axis)
    shape = list(sort.shape)
    shape[axis] = 1
    # Create a boolean array along strides of unique values
    strides = np.concatenate([np.zeros(shape=shape, dtype='bool'),
                                 np.diff(sort, axis=axis) == 0,
                                 np.zeros(shape=shape, dtype='bool')],
                                axis=axis).transpose(transpose).ravel()
    # Count the stride lengths
    counts = np.cumsum(strides)
    counts[~strides] = np.concatenate([[0], np.diff(counts[~strides])])
    counts[strides] = 0
    # Get shape of padded counts and slice to return to the original shape
    shape = np.array(sort.shape)
    shape[axis] += 1
    shape = shape[transpose]
    slices = [slice(None)] * ndim
    slices[axis] = slice(1, None)
    # Reshape and compute final counts
    counts = counts.reshape(shape).transpose(transpose)[slices] + 1

    # Find maximum counts and return modals/counts
    slices = [slice(None, i) for i in sort.shape]
    del slices[axis]
    index = np.ogrid[slices]
    index.insert(axis, np.argmax(counts, axis=axis))
    return sort[index]

# Ising model for binary image denoising

We want to be able to denoise binary images using the Ising model. For this purpose we use MCMC methods such as Gibbs sampling and Metropolis-Hastings.

In [4]:
def neighbourhood(M, coords, colour, n=4):
    """
    Calculates the the quantity:

        beta * same_colours

    where `same_colours` indicate the number
    of neighbours with the same colour as the
    input argument.

    Neighbourhood scheme:

        [[0, 1, 0],
         [1, 0, 1],
         [0, 1, 0]]
    """
    height, width = M.shape
    i, j = coords

    def neighbours(x, y, n=4):
        # Four neighbour scheme
        neighbours = np.array([(x-1, y), (x, y-1), (x+1, y), (x, y+1)])
        # Eight neighbour scheme
        if n >= 8:
            diagonal = np.array([(x-1, y-1), (x-1, y+1), (x+1, y-1), (x+1, y+1)])
            neighbours = np.vstack([neighbours, diagonal])
        # Twelve neighbour scheme
        if n >= 12:
            extremal = np.array([(x-2, y), (x, y-2), (x+2, y), (x, y+2)])
            neighbours = np.vstack([neighbours, extremal])
        return neighbours

    def boundary(index):
        i, j = index
        # Remove boundary points
        if i < 0 or j < 0:
            return False
        if i >= height or j >= width:
            return False
        return True

    positions = list(filter(boundary, neighbours(i, j, n)))

    neighbour_colours = [M[i, j] for i, j in np.array(positions)]
    same_colours = neighbour_colours.count(colour)

    return same_colours

### Gibbs sampling

In [5]:
def gibbs_ising(Y, n_iter, noise_var=0.3, beta=1):
    """
    Computes a binary output image based on
    input image Y. An Ising model is solved
    locally in order to denoise Y.
    """
    n, m = Y.shape
    X = np.random.randint(0, 2, (n, m))

    history = np.zeros((n_iter+1, n, m))
    history[0] = X

    # Generate a chain of length n_iter
    for k in range(n_iter):
        for i in range(n):
            for j in range(m):
                n_black = neighbourhood_ising(Y, (i, j), 0, beta)
                n_white = neighbourhood_ising(Y, (i, j), 1, beta)

                # Acceptance probability
                p = np.exp(beta*n_black)/(np.exp(beta*n_black) + np.exp(beta*n_white))
                U = np.random.uniform()
                if U < p:
                    X[i,j] = 1
                else:
                    X[i,j] = 0

        history[k+1] = X

    return X, history

### Metropolis-Hastings

In [6]:
def mh_sample(Y, n_iter, noise_var=0.3, beta=1):
    """
    Computes a sample output image based
    from the posterior based on input
    image Y. An Ising model is solved
    locally in order to provide a denoised
    sample of Y.
    """
    n, m = Y.shape
    X = np.random.randint(0, 2, (n, m))

    # Generate a chain of length n_iter
    for k in range(n_iter):
        for i in range(n):
            for j in range(m):
                x_candidate = 1 - X[i,j]
                #x_candidate = np.random.randint(0, 2)

                h           = - 1.0/(2*noise_var) * (Y[i,j] - X[i,j])**2
                h_candidate = - 1.0/(2*noise_var) * (Y[i,j] - x_candidate)**2

                d           = beta * neighbourhood(X, (i, j), X[i,j]) + h
                d_candidate = beta * neighbourhood(X, (i, j), x_candidate) + h_candidate

                # Acceptance probability
                p = np.exp(min(0, d_candidate - d))
                U = np.random.uniform()
                if U < p:
                    X[i,j] = x_candidate

    return X


### Image estimators

In [7]:
def map_estimate(Y, n_iter, T0= 4, noise_var=0.3, beta=1):
    """
    Computes a Maximum A Posteriori (MAP)
    estimate of the true image based on
    an Ising model with input image Y.
    The MAP estimate simulated annealing.
    """

    n, m = Y.shape
    X = np.random.randint(0, 2, (n, m))
    T = T0

    # Generate a chain of length n_iter
    for _ in range(n_iter):
        for i in range(n):
            for j in range(m):
                x_candidate = 1 - X[i,j]

                h = - 1.0/(2*noise_var) * (Y[i,j] - X[i,j])**2
                h_candidate = - 1.0/(2*noise_var) * (Y[i,j] - x_candidate)**2

                d           = neighbourhood_ising(X, (i, j), X[i,j], beta) + h
                d_candidate = neighbourhood_ising(X, (i, j), x_candidate, beta) + h_candidate

                # Acceptance probability
                p = np.exp(min(0, d_candidate - d)/T)
                U = np.random.uniform()
                if U < p:
                    X[i,j] = x_candidate

        T = T0*0.999**(n_iter)
    return X


def mpm_estimate(Y, m = 10, n_iter = 10, noise_var = 0.3, beta = 1):
    """
    Computes the Marginal Posterior Mode (MPM) estimate
    of the true image based on the noisy image Y.
    """
    X = np.array([mh_sample(Y, n_iter, noise_var, beta) for _ in range(m)])
    return mode(X, 0)

The Ising-based Metropolis-Hastings method is applied to the data, in this case a silhouette of a horse.

In [8]:
from skimage import measure
from skimage import data
from skimage.util import random_noise

# How do we estimate this for our model?
variance = 0.3

X = data.horse()
Y = random_noise(X, "gaussian", var=variance)
X = X.astype(int)

In [16]:
X_map = map_estimate(Y, 100)

In [0]:
X_mpm = mpm_estimate(Y, 10, 10)

In [0]:
result = [measure.mse(X,X_map), measure.mse(X,X_mpm)]

## Plots for the report

In [110]:
img = np.loadtxt("../cpp/data/noisy.txt").reshape(300, 465)

FileNotFoundError: [Errno 2] No such file or directory: '../cpp/data/noisy.txt'

In [19]:
imgs = np.array([np.loadtxt("../cpp/history/mt{}.txt".format(i)) for i in range(1, 11)]).reshape(-1, 300, 465)

In [33]:
plt.figure(figsize=(50, 15))
plt.imshow(mode(imgs), cmap="gray")

In [39]:
from skimage import measure
measure.compare_mse(mode(imgs), img)

0.025694267189762966

In [41]:
measure.compare_mse(np.mean(imgs, 0), img)

0.02205639976408302

In [42]:
from PIL import Image

In [84]:
from skimage.util import random_noise

In [172]:
def save_all(img, base_path):
    #noisy = random_noise(img, "localvar", mean=0, var=0.2)
    noisy = random_noise(img, "speckle", mean=0, var=0.025)
    plt.imsave(base_path + ".jpg", noisy)
    
    gray = np.mean(noisy, 2)
    
    plt.imsave(base_path + "-gray.jpg", gray, cmap="gray")
    np.savetxt(base_path + "-gray.txt", gray)
    
    for i, ch in enumerate(["r", "g", "b"]):
        plt.imsave(base_path + "-" + ch + ".jpg", noisy[:, :, i], cmap="gray")
        np.savetxt(base_path + "-" + ch + ".txt", noisy[:, :, i])

In [163]:
?random_noise

In [148]:
img = Image.open("/Users/jesperwohlerthansen/Downloads/lars.jpg").resize((480, 320))

In [170]:
plt.figure(figsize=(15, 15))

plt.imshow(random_noise(np.array(img)[:, :, 0], "speckle", mean=0, var=0.025), cmap="gray")

In [173]:
save_all(np.array(img), "/Users/jesperwohlerthansen/Education/Stochastic Simulation/cpp/data/lars-noisy")

In [209]:
grayscale = np.mean(np.array(img), 2)/255

In [214]:
from skimage import measure

def mse(comparison):
    for i in range(20):
        img = Image.open("/Users/jesperwohlerthansen/Education/Stochastic Simulation/cpp/history/history_{}.jpg".format(i))
        img = np.array(img).astype(np.float64)/255
        
        print(measure.compare_mse(img, comparison))

In [215]:
mse(grayscale)

0.0139364164936
0.0182940561834
0.0197848990484
0.0193588228397
0.0208907339436
0.020635433298
0.020304580214
0.0199562605573
0.0195514054201
0.0193315925276
0.0188907672174
0.018428567616
0.0201681254517
0.0208547759186
0.0209295397982
0.0209754271639
0.0208154262361
0.020583485129
0.020295829373
0.0201603160888
